In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd

from hp_pred.experiments import load_labelized_cases
from hp_pred.test_model import TestModel

optuna.logging.set_verbosity(optuna.logging.WARNING)

## Load and format data

In [6]:
# import the data frame and add the meta data to the segments

dataset_name = '30_s_dataset'
model_filename = 'xgb_30_s.json'


In [7]:
data = pd.read_parquet(f'data/datasets/{dataset_name}/cases', dtype_backend='pyarrow')
static = pd.read_parquet(f'data/datasets/{dataset_name}/meta.parquet')

data = data.merge(static, on='caseid')
test = data[data['split'] == 'test']
train = data[data['split'] == 'train'] # training set in only used to calibrate the baseline


## Evaluation

This section should take about 10 minutes to complete.

In [ ]:
tester = TestModel(
    test,
    train,
    [model_filename], # a list of model can be provided
    output_name='30_s_model',
)
tester.run(True, True)

## SHAP interpretations

In [ ]:
tester.compute_shap_value()
tester.plot_shap_values()
tester.group_shap_values()
tester.plot_shap_grouped()

## Exemple of cases


In [ ]:
# plot 3 random cases with the corresponding decision function

np.random.seed(1)
cases = np.random.choice(test['caseid'].unique(), 5, replace=False)
model =tester.model[0]
features_names = tester.features_names
for case in cases:
    raw_case = load_labelized_cases(dataset_path=Path(f'data/datasets/{dataset_name}/'), caseid=int(case))

    segment_data = test[test['caseid'] == case]

    segment_data = segment_data.dropna(subset=features_names)
    x_test = segment_data[features_names]
    y_pred = model.predict_proba(x_test)[:,1]

    plt.figure(figsize=(12, 5))
    plt.fill_between(raw_case.index.seconds /60, np.zeros(len(raw_case.index)), raw_case.label*100, label='label', alpha=0.2)
    # FIXME: raw_case.mbp might be NaN. fillna(0) ?
    plt.plot(raw_case.index.seconds /60, raw_case['mbp'])
    plt.hlines(65, raw_case.index.seconds[0]/60, raw_case.index.seconds[-1]/60, color='r', linestyle='--', label='IOH threshold')

    #plot in red point labeled as IOH
    plt.plot([t.hour * 60 + t.minute + t.second /60 for t in (segment_data[segment_data.label>0].time)] ,y_pred[segment_data.label>0]*100, 'r.', label='model decision function')
    plt.plot([t.hour * 60 + t.minute + t.second / 60 for t in (segment_data[segment_data.label==00].time)] ,y_pred[segment_data.label==0]*100, 'g.', label='model decision function')


    # plt.plot(segment_data.time / np.timedelta64(1, 's') /60,segment_data.time_before_IOH, 'x', label='model decision function')

    plt.xlabel('Time (min)')
    # plt.xlim([100, 120])
    # plt.xlim([235, 245])
    # plt.ylim([0, 100])
    plt.legend()
    plt.title(f'Case {case}')
    plt.grid()
    plt.show()


